In [ ]:
# @title Enter the `PyRosettaCluster` output directory
from pathlib import Path

# @markdown ##### E.g., enter `/content/data`. If the path begins with `/content/drive/MyDrive`, you will be asked to mount Google Drive
output_path = "/content/drive/MyDrive/pyrosettacluster-examples_example-1" # @param {type:"string"}
output_path = Path(output_path)

if str(output_path).startswith("/content/drive/MyDrive"):
    from google.colab import drive
    drive.mount("/content/drive")



In [ ]:
# @title Create the original `pixi` project
import os
import shutil
from pathlib import Path

# Install pixi
if not shutil.which("pixi"):
    !curl -fsSL https://pixi.sh/install.sh | sh
    os.environ["PATH"] = f"{os.getenv('PATH')}{os.pathsep}/root/.pixi/bin"
!pixi --version

# Clone GitHub repository
repo_path = Path.cwd() / "pyrosettacluster-examples"
if not repo_path.is_dir():
    !git clone https://github.com/klimaj/pyrosettacluster-examples.git {repo_path}

# Setup pixi project
manifest_path = repo_path / "envs" / "pixi" / "pixi.toml"
if (manifest_path.parent / "pixi.lock").exists():
    !pixi install --manifest-path {manifest_path} --frozen --no-progress
else:
    !pixi install --manifest-path {manifest_path} --no-progress

In [ ]:
# @title Run the original `PyRosettaCluster` simulation

num_total_tasks = 2000

def get_num_remaining_tasks(scorefile):
    if scorefile.exists():
        with scorefile.open("r") as f:
            num_complete_tasks = sum(1 for line in f)
            return num_total_tasks - num_complete_tasks
    else:
        return num_total_tasks

scratch_dir = Path("/content/scratch")
scorefile = output_path / "scores.json"
num_tasks = get_num_remaining_tasks(scorefile)
!cd {repo_path} && \
    pixi run --manifest-path {manifest_path} \
    python -m src.runners.example_1 \
    --output_path {output_path} \
    --scratch_dir {scratch_dir} \
    --num_tasks {num_tasks}

In [ ]:
# @title Print the highest and lowest energy decoys
!cd {repo_path} && \
    pixi run --manifest-path {manifest_path} \
    python -m src.plotting.analyze_example_1 \
    --original_scorefile {scorefile}